In [75]:
# Falcon specification: https://falcon-sign.info/falcon.pdf
# Falcon Python implementation: https://github.com/tprest/falcon.py
# NTRU Solve: https://tprest.github.io/pdf/slides/ntru-gen-pkc.pdf
# Lecture: https://simons.berkeley.edu/talks/generating-ntru-trapdoors

In [76]:
# global constants
n = 2^9
q=12289
sigma_min = 1.277833697 # see table 3.3 page 51
sigma_max = 1.8205
sigma = 165.736617183
beta = sqrt(34034726)

In [77]:
# use variables consistently
# to avoid clash of types, use variables x, X, y, Y, z, Z as described below
Zx.<x> = PolynomialRing(ZZ) 
#Zphi.<X> = Zx.quotient(x^4+1)
#Qy.<y> = PolynomialRing(QQ)
#Qphi.<Y> = Qy.quotient(y^4+1)
#Zq.<z> = PolynomialRing(Integers(7))
#ZQphi.<Z> = Zq.quotient(z^4+1)

In [78]:
# ============================ POLYNOMIAL UTILS ===================================

# algorythm 16 page 45, author: Maxim Pushkar
def Balance(f,q,n):
    g = list(((f[i] + q//2) % q) - q//2 for i in range(n))
    return f.parent()(g)

# formula 3.21 from page 28, author: Evgen Postulga
def Split(f,n):
    f0 = list(f[2*i+0] for i in range(n//2))
    f1 = list(f[2*i+1] for i in range(n//2))
    return f.parent()(f0), f.parent()(f1)  

# formula 3.22 from page 28, author: Evgen Postulga
def Merge(a,b):
    a = a.subs(x=x^2)
    b = x*b.subs(x=x^2)
    return a+b

# formula 3.6 from page 23, author: Evgen Postulga
def HermitianAdjointPoly(p, n):
    f=[p[0]]
    for i in range(1,n):    
        f.append(-p[n-i])
    return p.parent()(f) 

# formula 3.10 from page 24, author: Evgen Postulga
def InnerProduct(a,b,n):
    # return sum([a[i] * b[i] for i in range(n)])
    s=0
    for i in range(n):
        s=s+a[i]*b[i]
    return(s)

# formula 3.9 from page 24, author: Evgen Postulga
def EuclideanNorm(a,n):
    # return sqrt(InnerProduct(a,a,n))
    b=InnerProduct(a,a,n)
    return b^(1/2)

# formula 3.25 from page 30, author: Maxim Pushkar
def FieldNorm(f, n):
    f0, f1 = Split(f, n)
    iks = f.parent()([0, 1])
    return (f0^2 - iks * f1^2) % (iks^(n/2)+1)
    # return (f0^2-x*f1^2)%(x^(n/2)+1)

# NNT representation of polynomial f from Zq[x]/x^n+1, page 28, author Karina Ilchenko
def NTT(f, n, q):
    # Zp
    #g = f.change_ring(Integers(q))
    #return list(g(root[0]) for root in (x^n+1).roots(Integers(q)))    
    roots = (x^n + 1).roots(Integers(q))
    ans = [f.subs(x = i[0]) % q for i in roots]
    return ans


In [79]:
# ======================== Lattice Matrices ============================

# author: Evgen Postulga
def CyclicRotate(input, n):
    #return input[(len(input)-n):]+input[:(len(input)-n)]
    return input[-n:] + input[0:-n]

# author: Evgen Postulga
def PolyToCirculant(p, n):
    M=[]
    k=p.coefficients(sparse=False)
    while len(k)!=n:
        k.append(0)
    for i in range(n):
        m = CyclicRotate(k, i)
        M.append(m)
    return Matrix(M)

# author: Evgen Postulga
def CirculantToPoly(M):
    # Zx.<x> = PolynomialRing(ZZ)
    return Zx(list(M[0]))
    
# author: Evgen Postulga
def PolyToLattice4(p00, p01, p10, p11, n):
    M=[]
    p=[p00.coefficients(sparse=False),p01.coefficients(sparse=False),p10.coefficients(sparse=False),p11.coefficients(sparse=False)]
    for i in range(4):
        while len(p[i])!=n:
            p[i].append(0)
    for i in range(n):
        m1 = CyclicRotate(p[0], i)
        m2 = CyclicRotate(p[1], i)
        M.append(m1+m2)
    for i in range(n):
        m1 = CyclicRotate(p[2], i)
        m2 = CyclicRotate(p[3], i)
        M.append(m1+m2)
    return Matrix(M)

def PolyToLatticeFFT(p00, p01, p10, p11, n):
    M=[]
    p=[p00,p01,p10,p11]
    for i in range(4):
        while len(p[i])!=n:
            p[i].append(0)
    for i in range(n):
        m1 = CyclicRotate(p[0], i)
        m2 = CyclicRotate(p[1], i)
        M.append(m1+m2)
    for i in range(n):
        m1 = CyclicRotate(p[2], i)
        m2 = CyclicRotate(p[3], i)
        M.append(m1+m2)
    return M

# author: Evgen Postulga
def LatticeToPoly4(M, n_2):
    p00=M[0][:n]
    p01=M[0][n:]
    p10=M[n][:n]
    p11=M[n][n:]
    return p00, p01, p10, p11

# formula 3.7 page 23, author: Evgen Postulga
def HermitianAdjointMatrix(M, n2):
    a, c, b, d = LatticeToPoly4(M, n2/2)
    a, c, b, d = HermitianAdjointPoly(a, n2/2), HermitianAdjointPoly(c, n2/2),HermitianAdjointPoly(b, n2/2),HermitianAdjointPoly(d, n2/2)
    return PolyToLattice4(a, c, b, d, n2/2)

In [80]:
# =========================== RANDOM GENERATORS ============================

def UniformBits(k):
    return int.from_bytes(bytes(list(floor(uniform(0, 256)) for i in range(k / 8))), 'little')
    
def BaseSampler():
    u = UniformBits(72)
    z_0 = 0
    RCDT = [3024686241123004913666, 1564742784480091954050, 636254429462080897535, 199560484645026482916, 47667343854657281903, 8595902006365044063, 1163297957344668388, 117656387352093658, 8867391802663976, 496969357462633, 20680885154299, 638331848991, 14602316184, 247426747, 3104126, 28824, 198, 1, 0]
    for i in range(0, 18):
        z_0 = z_0 + int(u<RCDT[i]) 
    return z_0

def ApproxExp(x, ccs):
    C = [0x00000004741183A3,0x00000036548CFC06,0x0000024FDCBF140A,0x0000171D939DE045,0x0000D00CF58F6F84,0x000680681CF796E3,0x002D82D8305B0FEA,0x011111110E066FD0,0x0555555555070F00,0x155555555581FF00,0x400000000002B400,0x7FFFFFFFFFFF4800,0x8000000000000000]
    y = C[0]
    z = floor(2^63*x)
    for i in range(1, 13):
        y = C[i] - (z*y) >> 63
    z = floor(2^63*ccs)
    y = (z*y) >> 63
    return y

def BerExp(x, ccs):
    s = floor(x/ln(2))
    r = x - s*ln(2)
    s = min(s, 63)
    z = (2*ApproxExp(r, ccs) - 1) >> s
    for i in range(56, -8, -8):
        p = UniformBits(8)
        w = p - ((z >> i) & 0xFF)
        if int(w) == 0:
            break
    return int(w < 0)

def SamplerZ(mu, sigma, sigmamin, sigmamax):
    r = mu - int(floor(mu))
    ccs = sigmamin/sigma
    while True:
        z_0 = BaseSampler()
        b = UniformBits(8)&0x1
        z = b + (2*b-1)*z_0
        x = (z-r)^2/2/sigma^2 - z_0^2/2/sigmamax^2
        if BerExp(x, ccs) == 1:
            return z + int(floor(mu))

In [81]:
print("SamplerZ =", SamplerZ(mu=0, sigma=1.17 * sqrt(12289 / 8192), sigmamin=1.277833697, sigmamax=1.8205))

SamplerZ = 0


In [82]:
# ============================ GENERATE f, F, g, G =============================

# algorithm 7, page 35, author Karina Ilchenko
# also https://www.h2020prometheus.eu/sites/default/files/2019-11/More%20Efficient%20Algorithms%20for%20the%20NTRU%20Key%20Generation%20using%20the%20Field%20Norm.pdf 
# page 10
def Reduce(f, g, F, G, n):
    
    T = Zx.change_ring(QQ).quotient(x^n+1) 
    
    iks = f.parent()([0, 1])
    f_star = HermitianAdjointPoly(f, n)
    g_star = HermitianAdjointPoly(g, n)
    while True:
        num = F*f_star + G*g_star
        num = T(num)
        den = f*f_star + g*g_star
        den = 1 / T(den)
        res = num * den
        k = Zx([int(round(elt)) for elt in res])
        F = F - k*f 
        G = G - k*g
        if all(elt == 0 for elt in k):
            break

    return f, g, F, G

# algorithm 6, page 35, author Karina Ilchenko
# also https://tprest.github.io/pdf/slides/ntru-gen-pkc.pdf
# and https://www.h2020prometheus.eu/sites/default/files/2019-11/More%20Efficient%20Algorithms%20for%20the%20NTRU%20Key%20Generation%20using%20the%20Field%20Norm.pdf
def NTRUSolve(f, g, n, q):
    if n == 1:
        # u, v are numbers
        gcd_, u, v = xgcd(f[0], g[0])
        #print("gcd", u * f + v * g, "\n")
        if gcd_ != 1:
            return None, None, False
        F, G = -v*q, u*q
        #print("F1, G1", F / q, G / q)
        return F, G, True
    else:
        # ▷ f′, g′, F′, G′ ∈ Z[x]/(x^n/2 + 1)
        # ▷ N as defined in either (3.25) or (3.26)
        f_ = FieldNorm(f, n) 
        g_ = FieldNorm(g, n) 
        #print(n//2, f_, g_, sep="\n")
        F_, G_, flag = NTRUSolve(f_, g_, n//2, q)
        if flag:
            F = F_.subs(x=x^2) * g.subs(x=-x) 
            G = G_.subs(x=x^2) * f.subs(x=-x)
            #print("F, G", F, G, sep="\n")
            f, g, F, G = Reduce(f, g, F, G, n)
            return F % (x^n +1), G % (x^n +1), flag
        else:
            return F_, G_, flag

# algorithm 6, page 35, author Karina Ilchenko
# also https://tprest.github.io/pdf/slides/ntru-gen-pkc.pdf
def NTRUGen(q, n):

    # f = ZQphi(list(sum(SamplerZ(0, 1.17 * sqrt(q / 8192), sigma_min, sigma_max) for j in range(1, 4096 / n + 1)) for i in range(n)))
    # g = ZQphi(list(sum(SamplerZ(0, 1.17 * sqrt(q / 8192), sigma_min, sigma_max) for j in range(1, 4096 / n + 1)) for i in range(n)))
    # h = g / f
    def gen_poly(n, q):
        
        def D(mu=0):
            z = 0
            for i in range(1, 4096/n + 1):
                sigma_star = 1.17 * sqrt(q / 8192)
                sigmamin, sigmamax = 1.277833697, 1.8205
                zi = SamplerZ(mu, sigma_star, sigmamin, sigmamax)
                z += zi
            return z

        f = [0] * n
        g = [0] * n
        for i in range(n):
            f[i] = D()
            g[i] = D()
        f = Zx(f) % (x^n+1)
        g = Zx(g) % (x^n+1)
        return f, g
    
    def gs_norm(f, g, q, n):
        T = Zx.change_ring(QQ).quotient(x^n+1) 
        # Using (3.9) with (3.8) or (3.10)    
        f_star = HermitianAdjointPoly(f, n)
        g_star = HermitianAdjointPoly(g, n)
        first = EuclideanNorm([*g.coefficients(sparse=False), *(-f).coefficients(sparse=False)], n)
        s1 = (q * T(f_star)) / T((f*f_star + g*g_star))
        s2 = (q * T(g_star)) / T((f*f_star + g*g_star))
        second = EuclideanNorm(list(s1) + list(s2), n)
        gamma = max(first, second)
        return gamma

    while True:
        while True:
            while True:

                f, g = gen_poly(n, q)
                # print(f, g, sep="\n")

                if gs_norm(f, g, q, n) > (1.17 ** 2) * q:
                    # print("restart norm\n")
                    continue
                break

            if  0 in NTT(f, n, q):
                # print("restart ntt\n")
                continue
            break
                
        F, G, flag = NTRUSolve(f, g, n, q)
        
        if not flag:
            # print("restart solve")
            continue
        else:
            F, G = F % (x^n +1), G % (x^n +1)
            F = Zx([int(coef) for coef in F.coefficients(sparse=False)])
            G = Zx([int(coef) for coef in G.coefficients(sparse=False)])
            tt = Zx.change_ring(Integers(q)).quotient(x^n+1)
            pk = Zx(lift(tt(g) / tt(f))) 
            print("f =", f)
            print("g =", g)
            print("F =", F)
            print("G =", G)
            print("(f*G - g*F) % (x^n + 1) == q", (f*G - g*F) % (x^n + 1) == q)
            print("Public Key:", pk)
            break
            
    return f, g, F, G, pk
      

In [83]:
# ============================ FAST FOURIER TRANSFORM =========================
# Accordint to page 27, polynomials must be from Q[y]/(y^n+1)

# roots of monic polynomial https://github.com/tprest/falcon.py/blob/master/scripts/generate_constants.sage
def Omega(n):
    #if n == 2:
    #    return [I, -I]
    #else:
    #    return sum([[sqrt(elt), - sqrt(elt)] for elt in Omega(n // 2)], [])
    phi4 = cyclotomic_polynomial(4)
    phi_n = phi4.complex_roots()
    phi_n.reverse()
    k = 2
    while k != n:
        phi_n = sum([[sqrt(elt), - sqrt(elt)] for elt in phi_n], [])
        k = 2*k
    return phi_n

# formula 3.18, page 27, author: Evgen Postulga
def FFT(f):
    n = len(list(f))
    return [lift(f).subs(root) for root in Omega(n)]

# algorithm 1, page 29, author: Evgen Postulga
def splitfft(f):
    n = len(f)
    w = Omega(n)
    f0 = [0] * (n // 2)
    f1 = [0] * (n // 2)
    for i in range(n // 2):
        f0[i] = 0.5 * (f[2 * i] + f[2 * i + 1])
        f1[i] = 0.5 * (f[2 * i] - f[2 * i + 1]) / w[2 * i]
    return f0, f1

# algorithm 2, page 29, author: Evgen Postulga
def mergefft(f0, f1):
    n = 2 * len(f0)
    w = Omega(n)
    f = [0] * n
    for i in range(n // 2):
        f[2 * i + 0] = f0[i] + w[2 * i] * f1[i]
        f[2 * i + 1] = f0[i] - w[2 * i] * f1[i]
    return f

# https://github.com/tprest/falcon.py/blob/master/fft.py
def invFFT(f_fft):
    n = len(f_fft)
    if (n > 2):
        f0_fft, f1_fft = splitfft(f_fft)
        f0 = invFFT(f0_fft)
        f1 = invFFT(f1_fft)

        f = n*[0]
        for i in range(n//2):
            f[2*i+0] = f0[i]
            f[2*i+1] = f1[i] 

    elif (n == 2):
        f = [0, 0]
        f[0] = f_fft[0].real()
        f[1] = f_fft[0].imag()
    return f

def mul_fft(a,b):
    return list(a[i] * b[i] for i in range(len(a)))
def div_fft(a,b):
    return list(a[i] / b[i] for i in range(len(a)))
def add_fft(a,b):
    return list(a[i] + b[i] for i in range(len(a)))
def sub_fft(a,b):
    return list(a[i] - b[i] for i in range(len(a)))
def neg_fft(a):
    return list(     - a[i] for i in range(len(a)))

In [84]:
# FFT Test 1: test that FFT and invFFT are inverse operations
Qy.<y> = PolynomialRing(QQ)
Qphi.<Y> = Qy.quotient(y^4+1) 
f = Y^3-5*Y+1; print(f)
g = invFFT(FFT(f)); print(Qphi(g))

Y^3 - 5*Y + 1
Y^3 - 5*Y + 1


In [85]:
# FFT Test 2: operations +-*/ over polynomials correspond to per coordinate +-*/
f = Y^3-5*Y+1; g = Y^2-7*Y+3
print("Multiplication: ", mul_fft(FFT(f), FFT(g)), " ===", FFT(f * g))
print("Division: ",       div_fft(FFT(f), FFT(g)), " ===", FFT(f / g))
print("Addition: ",       add_fft(FFT(f), FFT(g)), " ===", FFT(f + g))
print("Subtraction: ",    sub_fft(FFT(f), FFT(g)), " ===", FFT(f - g))
print("Negation: ",       neg_fft(FFT(f)),         " ===", FFT(-f))

Multiplication:  [-4.84924240491749 + 18.3223304703363*I, 24.8492424049175 + 53.6776695296637*I, -4.84924240491749 - 18.3223304703363*I, 24.8492424049175 - 53.6776695296637*I]  === [-4.84924240491750 + 18.3223304703363*I, 24.8492424049175 + 53.6776695296637*I, -4.84924240491750 - 18.3223304703363*I, 24.8492424049175 - 53.6776695296637*I]
Division:  [0.901653699819546 - 0.375883187601447*I, 0.593380278225410 - 0.0883091804069174*I, 0.901653699819546 + 0.375883187601447*I, 0.593380278225410 + 0.0883091804069174*I]  === [0.901653699819545 - 0.375883187601446*I, 0.593380278225410 - 0.0883091804069174*I, 0.901653699819545 + 0.375883187601446*I, 0.593380278225410 + 0.0883091804069174*I]
Addition:  [-5.19238815542512 - 6.77817459305202*I, 13.1923881554251 + 8.77817459305202*I, -5.19238815542512 + 6.77817459305202*I, 13.1923881554251 - 8.77817459305202*I]  === [-5.19238815542512 - 6.77817459305202*I, 13.1923881554251 + 8.77817459305202*I, -5.19238815542512 + 6.77817459305202*I, 13.192388155425

In [ ]:
# ======================== Key Pair Generation ============================

# binary tree implementation
class Node:
    def __init__(self, value):
        self.value = value
        self.leftchild = 0
        self.rightchild = 0
        self.tree = [self.value,self.leftchild,self.rightchild]
        
    def update_tree(self):
        self.tree = [self.value,self.leftchild,self.rightchild]
        
    def __str__(self):
        return '[' + str(self.value) + ',' + str(self.leftchild) + ','  + str(self.rightchild) + ']' 
    
    def __repr__(self):
        return '[' + str(self.value) + ',' + str(self.leftchild) + ','  + str(self.rightchild) + ']'
    
    def print_tree(self, pref=""):
        leaf = "|—————> "
        top = "|_______"
        son1 = "|       "
        son2 = "        "
        width = len(top)

        a = ""
        if (self.value * x^0).degree() and self.leftchild:
            if (pref == ""):
                a += pref + str(self.value) + "\n"
            else:
                a += pref[:-width] + top + str(self.value) + "\n"
            try:
                a += self.leftchild.print_tree(pref + son1)
                a += self.rightchild.print_tree(pref + son2)
            except:
                pass
            return a
        else:
            return (pref[:-width] + leaf + str(self.value) + "\n")

def normalize_tree(tree, sigma):
    if (tree.value * x^0).degree() and tree.leftchild is not None:
        normalize_tree(tree.leftchild, sigma)
        normalize_tree(tree.rightchild, sigma)
    else:
        tree.value = sigma / sqrt(tree.value.real())
        tree.update_tree()

# todo: Kateryna Makowetska, please update this function
def LDL(g00, g01, g10, g11, n):
    D00 = g00
    g100 = FFT(g10,n)
    g000 = FFT(g00,n)
    L10 = div_fft(g100,g000)
    L10_star = HermitianAdjointPoly(invFFT2(L10),n)
    D11 = sub_fft(FFT(g11,n), mul_fft(mul_fft(L10,FFT(L10_star,n)),g000))
    L = PolyToLattice4(1*x^0,0*x^0,Qx(invFFT2(L10)),1*x^0,n)
    D = PolyToLattice4(D00,0*x^0,0*x^0,Qx(invFFT2(D11)),n)
    return L,D

# todo: Kateryna Makowetska, please update this function
def ffLDL(G):

    # n can be identified from matrix size
    n = G.nrows() // 2

    Qy.<y> = PolynomialRing(QQ)
    Qphi.<Y> = Qy.quotient(y^n+1)

    L, D = LDL(Qphi(g00), Qphi(g01), Qphi(g10), Qphi(g11), n)
    L10 = Qx(LatticeToPoly4(L,n)[2])
    tree = Node(L10)
    D00 = Qx(LatticeToPoly4(D,n)[0])
    D11 = Qx(LatticeToPoly4(D,n)[3])
    if n == 2:
        tree.leftchild = FFT(D00)[0]
        tree.rightchild = FFT(D11)[0]
        tree.update_tree()
        #print(tree)
        return tree
    else:
        d00,d01 = split_fft(FFT(D00,n))
        d10,d11 = split_fft(FFT(D11,n))
        d01_star = HermitianAdjointPoly(invFFT2(d01),n//2)
        d11_star = HermitianAdjointPoly(invFFT2(d00),n//2)
        tree.leftchild = ffLDL(invFFT(d00),invFFT2(d01),d01_star,invFFT(d00),n//2)
        tree.rightchild = ffLDL(invFFT(d01),invFFT2(d11),d11_star,invFFT(d10),n//2)
    return tree

# algorithm 4 page 33, author: 
# todo: Kateryna Makowetska, please update this function
def KeyGen(sigma, q, n):

    # generate keys
    f, g, F, G, pk = NTRUGen(q, n)

    # construct 2n X 2n matrix Bhat
    Qy.<y> = PolynomialRing(QQ)
    Qphi.<Y> = Qy.quotient(y^n+1)
    f_fft, g_fft, F_fft, G_fft = FFT(Qphi(f.subs(x=y))), FFT(Qphi(g.subs(x=y))), FFT(Qphi(F.subs(x=y))), FFT(Qphi(G.subs(x=y)))
    B = Matrix(PolyToLatticeFFT(g_fft, neg_fft(f_fft), G_fft, neg_fft(F_fft), 4))

    # construct 2n X 2n matrix Bhat^adjoint
    fa, ga, Fa, Ga = HermitianAdjointPoly(f, n), HermitianAdjointPoly(g, n), HermitianAdjointPoly(F, n), HermitianAdjointPoly(G, n)
    fa_fft, ga_fft, Fa_fft, Ga_fft = FFT(Qphi(fa.subs(x=y))), FFT(Qphi(ga.subs(x=y))), FFT(Qphi(Fa.subs(x=y))), FFT(Qphi(Ga.subs(x=y)))
    Ba = Matrix(PolyToLatticeFFT(ga_fft, neg_fft(fa_fft), Ga_fft, neg_fft(Fa_fft), 4))

    # construct 2n X 2n matrix G
    G = B * Ba

    # build Falcon tree
    T = ffLDL(G)    
    normalize_tree(T, sigma)

    # return the keys
    return B, T, pk

In [86]:
# ========================== SIGN / VERIFY ==============================

# algorithm 3 page 31, author: Maxim Pushkar
# pip3 install pycryptodome
from Crypto.Hash import SHAKE256
def HashToPoint(noise, message, q, n):
    k = floor(2^16 / q)
    c = n*[0]
    ctx = SHAKE256.new()
    ctx.update(noise)
    ctx.update(message)
    i = 0
    while i < n:
        t = int.from_bytes(ctx.read(2), byteorder='big')
        if t < k * q:
            c[i] = t % q
            i += 1
    return Xz(c)

# algorithm 11 page 40, author Kateryna Makowetska
def ffSampling(t0_, t1_, T, sigmamin, sigmamax, q, n):
    # _ means constant , __ means ' 
    if n == 1:
        #print('t0 t1',t0_,t1_)
        #print('TT',T,n)
        sigma = T
        #print("mu1", t0_[0].real())
        #print("sigma",float(sigma.real()))
        #print("mu2,", t1_[0].real())
        
        z0_ = SamplerZ(t0_[0].real(),float(sigma.real()),sigmamin, sigmamax)
        z1_ = SamplerZ(t1_[0].real(),float(sigma.real()),sigmamin, sigmamax)
        #print('SAMPLERZ',z0_,z1_)
        #print("==================")
        return [z0_],[z1_]
    #print('T',T,n)
    l = T.value
    T0 = T.leftchild
    T1 = T.rightchild
    t1 = split_fft(t1_)
    z1 = ffSampling(*t1, T1, sigmamin, sigmamax, q, n/2)
    z1_ = merge_fft(z1)
    t0__ = add_fft(t0_, mul_fft(sub_fft(t1_,z1_),l))
    t0 = split_fft(t0__)
    z0 = ffSampling(*t0, T0, sigmamin, sigmamax, q, n/2)
    z0_ = merge_fft(z0)
    return z0_,z1_

# algorithm 10, page 39, author: Maxim Pushkar
def Sign(message, B, T, beta, sigma_min, sigma_max, q, n): 
    # bound = int(beta^2)
    #r = UniformBits(320)
    r = b'befhehfjfn'
    c = HashToPoint(r, message, q, n)
    
    Qy.<y> = PolynomialRing(QQ)
    Qphi.<Y> = Qy.quotient(y^n+1)
    
    g, f, G, F = LatticeToPoly4(B, 4)  # наверное
    f, F = neg_fft(f), neg_fft(F)
    
    t0 = mul_fft(FFT(Qphi(c.subs(x=y))), f) 
    t1 = mul_fft(FFT(Qphi(c.subs(x=y))), f)  #todo
    
    t = vector(t0 + t1)
    
    # альтернативная версия + сверить обе
    
    while True:
        z1,z2 = ffSampling(t0, t1, T, sigma_min, sigma_max, q, n)
        s = (t - vector(z1+z2)) * B
        
        if EuclideanNorm(s,2*n) <= beta:
            s1, s2 = invFFT(s[:n]), invFFT(s[n:])
            # s = Compress(s2, ...)
            s = s2
            return r, s

# algorithm 16 page 45, author: Maxim Pushkar
def Verify(message, r, s2, pk, beta, q, n):

    c = HashToPoint(r, message, q, n)
    s1 = Balance((c - s2*pk) % (x^n + 1), q, n)  
    if (EuclideanNorm(s1,n)^2 + EuclideanNorm(s2,n)^2) <= beta^2:
        print("signature accepted")
        return True
    
    print("signature rejected")
    return False

In [87]:
# =========================== SIGN - VERIFY ===========================

# 1. generate keys
#f, g, F, G, pk = NTRUGen(q=17, n=8)
f, g, F, G, pk = NTRUGen(q=12289, n=4)

# 2. convert to FFT
Qy.<y> = PolynomialRing(QQ)
Qphi.<Y> = Qy.quotient(y^n+1)
f_fft, g_fft, F_fft, G_fft = FFT(Qphi(f.subs(x=y))), FFT(Qphi(g.subs(x=y))), FFT(Qphi(F.subs(x=y))), FFT(Qphi(G.subs(x=y)))
B = Matrix(PolyToLatticeFFT(g_fft, neg_fft(f_fft), G_fft, neg_fft(F_fft), 4))

f = 334*x^3 + 261*x^2 + 300*x + 308
g = 382*x^3 + 247*x^2 + 290*x + 311
F = 6*x^3 - 314*x^2 - 109*x + 12
G = 11*x^3 - 335*x^2 - 109*x + 42
(f*G - g*F) % (x^n + 1) == q True
Public Key: 10792*x^3 + 7248*x^2 + 7381*x + 2754


In [ ]:
# 2. Sign
r, s2 = Sign(b'hello', B, T, beta=sqrt(34034726), sigma_min=1.277833697, sigma_max=1.8205, q=12289, n=4)
print("Signature: noise =", r, ", s2 =", s2)

In [91]:
# 3. Verify
print(Verify(b'hello', r, s2 , pk, beta=sqrt(34034726), q=12289, n=4))